In [1]:
import pandas as pd
import numpy as np
import jieba.analyse
import collections
from gensim import corpora, models
from sklearn.feature_extraction.text import HashingVectorizer
import operator
from sklearn.preprocessing import OneHotEncoder

In [2]:
data_file = 'F:\\标注比赛\\大数据应用分类标注-选手\\apptype_train.dat'
label_id = 'F:\\标注比赛\\大数据应用分类标注-选手\\apptype_id_name.txt'

In [103]:
stop_word = [] 
with open('F:\标注比赛\stop_words.txt') as f: 
    line = f.readline() 
    while line: 
        line = line.split('\n') 
        stop_word.append(line[0]) 
        line = f.readline() 
    stop_word = set(stop_word)

def data_process(data_file, model='test'): 
    enc = OneHotEncoder()
    i = -1  
    with open(data_file, encoding='utf-8') as f: 
        line = f.readline() 
        all_words = [] 
        sentenses = [] 
        app_id_list = [] 
        length = []
        data = []
        data_sentense = []
        while line: 
            i = i + 1
            #if i in youxi:
            #    line = f.readline()
            #    continue
            desc_words = []
            word_list = []
            line = line.split('\n')
            line = line[0].split('\t')
            if model == 'train':   
                two_id = line[1].split('|')
                app_id = (int(two_id[0]) // 100)
                #if app_id != 1401 and app_id != 1409:
                #    line = f.readline()
                #    continue
                app_id_list.append(app_id)
            app_desc = line[-1]
            word_line = list(jieba.analyse.extract_tags(app_desc, withWeight=False, allowPOS=('n', 'vn')))
            #word_line = list(jieba.cut(app_desc))
            for word in word_line:
                if word not in stop_word:#and word not in ['qingkan520','www','com','http']:
                    desc_words.append(word)
                    all_words.append(word)
            #if app_id == 21:
            #    print(line)
            #    print(desc_words)
            #    break
            line = f.readline()

            sentenses.append(desc_words)
            
            #i = i+ 1
            #if i == 1000:
            #    break
#        counter = collections.Counter(all_words)
#        word_dict = counter.most_common(30000)
#        dict_list = [x[0] for x in word_dict]
#        dictionary = {}
#        for i in range(len(dict_list)):
#            dictionary[dict_list[i]] = i

        
        num_doc_train = len(sentenses)
        
#        for queue in sentenses:
#            for i in range(len(queue)):
#                data_tmp = dictionary.get(queue[i])

#                if not data_tmp:
#                    continue
#                data_sentense.append(data_tmp)
#            length.append(len(data_sentense))
#            data.append(data_sentense)
#        data_len = np.mean(length)
        
#        for i in range(num_doc_train):
#            if length[i] >= data_len:
#                data[i] = data[i][:data_len]
#            else:
#                data[i] = data[i]+[0 for x in range(data_len-length[i])]
#    label = enc.fit_transform(np.array(app_id_list).reshape(-1, 1)).toarray()
#    return data, data_len, label
        dictionary = corpora.Dictionary(sentenses)

        dictionary.filter_extremes(no_below=100, no_above=0.2, keep_n=2000)
        dictionary.compactify()

        corpus = [dictionary.doc2bow(test) for test in sentenses]

        tfidf = models.TfidfModel(corpus)
        corpus_tfidf = tfidf[corpus]
        return corpus, dictionary, num_doc_train, app_id_list

In [91]:
def LDA_model(corpus_tfidf, dictionary, num_doc_train):
    lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=2, chunksize=100, passes=2, iterations=5,
                                minimum_probability=0.01, random_state=1)
    topic = []
    for i in range(num_doc_train):
        topic_id = lda_model[corpus_tfidf[i]]
        topic_id.sort(key=operator.itemgetter(1))
        topic.append(topic_id[-1][0])
    print(lda_model.print_topics())
    print(lda_model.log_perplexity(corpus_tfidf))
    return topic

In [92]:
def label_compare(topic, app_id_list):
    #print(sorted(app_id_list))
    return

In [93]:
corpus_tfidf, dictionary, num_doc_train, app_id_list = data_process(data_file, 'train')
topic = LDA_model(corpus_tfidf, dictionary, num_doc_train)
label_compare(topic, app_id_list)

[(0, '0.015*"查询" + 0.011*"生活" + 0.010*"APP" + 0.009*"实时" + 0.009*"平台" + 0.008*"定位" + 0.008*"通过" + 0.008*"出行" + 0.007*"新增" + 0.007*"及"'), (1, '0.011*"微信" + 0.010*"视频" + 0.009*"文件" + 0.008*"自动" + 0.008*"一键" + 0.008*"红包" + 0.008*"设置" + 0.007*"游戏" + 0.007*"下载" + 0.007*"安全"')]
-6.416588429290033


In [95]:
df = pd.DataFrame(np.array([topic,app_id_list]).transpose(), columns=['predict', 'label'])
df_list = []
for i in range(2):
    df_list.append(df.loc[df.predict==i])


In [96]:
df_list[0]['label'].value_counts()

1401    1502
1409    1284
Name: label, dtype: int64

In [97]:
df_list[1]['label'].value_counts()

1409    3783
1401     260
Name: label, dtype: int64

In [78]:
df_list[2]['label'].value_counts()

IndexError: list index out of range

In [79]:
df_list[3]['label'].value_counts()

IndexError: list index out of range

In [80]:
df_list[4]['label'].value_counts()

IndexError: list index out of range

In [27]:
df_list[5]['label'].value_counts()

1409    422
1401    181
Name: label, dtype: int64

In [28]:
pd.DataFrame(np.array(app_id_list).transpose())[0].value_counts(ascending=False)

1409    5067
1401    1762
Name: 0, dtype: int64

In [190]:
youxi = []
for i in range(len(app_id_list)):
    if topic[i] == 2 and app_id_list[i] == 0:
        youxi.append(i)


In [ ]:
jiaoyu_and_licai = []
for i in range(len(app_id_list)):
    if topic[i] == 1:
        jiaoyu_and_licai.append(i)

In [217]:
i = 0
with open( 'F:\\标注比赛\\大数据应用分类标注-选手\\apptype_train.dat', encoding='utf-8') as f:
        line = f.readline()
        while line:
            if i in youxi:
                print(line)
                break
            i = i + 1
            line = f.readline()

00000777CE5B5AA5C1AC94DB8EABE0AC	140203	《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。性状本游戏为真正集换式卡牌游戏TCG，随着玩家的深入而不断释放独特魅力。功能主治锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。不良反应经临床实验，对智商90以下者的智商和自信有降低和灭杀反应。注意事项使用本游戏时请务必仔细阅读游戏内的新手入门，同时请参考不良反应。试用后请自行判断自己是否适合继续使用。更新内容《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。性状本游戏为真正集换式卡牌游戏TCG，随着玩家的深入而不断释放独特魅力。功能主治锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。不良反应经临床实验，对智商90以下者的智商和自信有降低和灭杀反应。注意事项使用本游戏时请务必仔细阅读游戏内的新手入门，同时请参考不良反应。试用后请自行判断自己是否适合继续使用。



In [332]:
dictionary.save('F:\\标注比赛\\大数据应用分类标注-选手\\ths_dict.dict')  # 保存生成的词典
#dictionary = corpora.Dictionary.load('ths_dict.dict')  # 加载

corpus_tfidf.save("F:\\标注比赛\\大数据应用分类标注-选手\\ths_tfidf.model")#保存成model格式
#corpus_tfidf = models.TfidfModel.load("ths_tfidf.model")#加载



e:\python\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
e:\python\lib\site-packages\gensim\interfaces.py:96: UserWarning: corpus.save() stores only the (tiny) iteration object in memory; to serialize the actual corpus content, use e.g. MmCorpus.serialize(corpus)
  "corpus.save() stores only the (tiny) iteration object in memory; "
e:\python\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [55]:
help(models.LdaModel)

Help on class LdaModel in module gensim.models.ldamodel:

class LdaModel(gensim.interfaces.TransformationABC, gensim.models.basemodel.BaseTopicModel)
 |  Train and use Online Latent Dirichlet Allocation (OLDA) models as presented in
 |  `Hoffman et al. :"Online Learning for Latent Dirichlet Allocation" <https://www.di.ens.fr/~fbach/mdhnips2010.pdf>`_.
 |  
 |  Examples
 |  -------
 |  Initialize a model using a Gensim corpus
 |  
 |  .. sourcecode:: pycon
 |  
 |      >>> from gensim.test.utils import common_corpus
 |      >>>
 |      >>> lda = LdaModel(common_corpus, num_topics=10)
 |  
 |  You can then infer topic distributions on new, unseen documents.
 |  
 |  .. sourcecode:: pycon
 |  
 |      >>> doc_bow = [(1, 0.3), (2, 0.1), (0, 0.09)]
 |      >>> doc_lda = lda[doc_bow]
 |  
 |  The model can be updated (trained) with new documents.
 |  
 |  .. sourcecode:: pycon
 |  
 |      >>> # In practice (corpus =/= initial training corpus), but we use the same here for simplicity.
 |    

In [66]:
b = [1,2,3,4,5]
b[:-1]

[1, 2, 3, 4]